# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,college,64.8569,-147.8028,-17.18,78,75,0.00,US,1707250351
1,1,blackmans bay,-43.0167,147.3167,12.07,82,97,2.05,AU,1707250351
2,2,bilibino,68.0546,166.4372,-34.95,100,100,0.68,RU,1707250352
3,3,ribeira grande,38.5167,-28.7000,17.10,94,75,13.89,PT,1707250353
4,4,pathein,16.7833,94.7333,22.58,67,0,4.51,MM,1707250354
...,...,...,...,...,...,...,...,...,...,...
569,569,albesti,47.7000,27.0667,8.57,73,100,1.77,RO,1707250791
570,570,maloye ganyushkino,46.6000,49.2667,-0.39,68,28,10.01,KZ,1707250791
571,571,nouadhibou,20.9310,-17.0347,23.99,38,49,5.66,MR,1707250793
572,572,semey,50.4111,80.2275,-8.77,78,85,6.04,KZ,1707250793


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
# Code to capture output of this cell and store it
# Does not display the output to keep notebook clean
# %%capture --no-display

# Code below is from Xpert learning assistant. Why would(n't) we use GeoDataFrame? When is it appropriate/necessary?
# Convert df to a GeoDataFrame (gdf).
# city_gdf = gpd.GeoDataFrame(city_data_df, geometry = gpd.points_from_xy())

In [4]:
%%capture --no-display

# Configure the map plot
# 6-3, activity 07
mat_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat", 
    geo=True, 
    tiles = "OSM", 
    frame_width = 800, 
    frame_height = 600,
    size = "Humidity",
    color = "City")

# Display the map
mat_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [34]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_df = city_data_df.loc[
    (city_data_df["Humidity"] < 85) &
    (city_data_df["Max Temp"]>0) & 
    (city_data_df["Max Temp"]<=15) &
    (city_data_df["Lat"]>= -22.5)]
ideal_df

# Drop any rows with null values
ideal_df_dropped = ideal_df.dropna()

# Display sample data
ideal_df_dropped

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
5,5,bethel,41.3712,-73.4140,6.05,48,0,4.12,US,1707250354
6,6,kaspiyskiy,45.3929,47.3707,0.84,68,61,7.50,RU,1707250355
19,19,eregli,37.5133,34.0467,8.52,47,5,3.40,TR,1707250365
23,23,hamilton,39.1834,-84.5333,9.77,59,0,2.57,US,1707249906
44,44,stanley,54.8680,-1.6985,3.77,74,20,4.63,GB,1707250383
...,...,...,...,...,...,...,...,...,...,...
550,550,chardara,41.2547,67.9692,11.30,36,100,9.14,KZ,1707250777
551,551,tokmok,42.8417,75.3016,1.97,48,78,3.60,KG,1707250778
562,562,klipphausen,51.0833,13.5333,10.40,77,20,9.26,DE,1707250786
566,566,moapa valley,36.5805,-114.4703,10.86,68,100,5.26,US,1707250714


### Step 3: Create a new DataFrame called `hotel_df`.

In [35]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity

# Copy the ideal_df_dropped 
copy_df = ideal_df_dropped.copy()

# Create new df with just city, coordinates, humidity, and country
#    by dropping city id, temp, cloudiness, wind speed, and date.
hotel_df = copy_df.drop(["City_ID","Max Temp","Cloudiness","Wind Speed","Date"], axis =1)

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Lat,Lng,Humidity,Country,Hotel Name
5,bethel,41.3712,-73.4140,48,US,
6,kaspiyskiy,45.3929,47.3707,68,RU,
19,eregli,37.5133,34.0467,47,TR,
23,hamilton,39.1834,-84.5333,59,US,
44,stanley,54.8680,-1.6985,74,GB,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [47]:
# Set parameters to search for a hotel

# Set parameters for limit, radius, filters, bias, and geoapify_key
limit = 20
radius = 10000
categories = f"accommodation.hotel"
geoapify_key

# Parameters dictionary
params = {"limit": limit,
         "apiKey": geoapify_key,
         "categories":categories}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
bethel - nearest hotel: Hampton Inn Danbury
kaspiyskiy - nearest hotel: Гостевой двор
eregli - nearest hotel: No hotel found
hamilton - nearest hotel: North Vista Manor
stanley - nearest hotel: Hotel 52
merano - nearest hotel: Meranerhof
arnsberg - nearest hotel: Hotel Sunderland
naze - nearest hotel: ビジネス旅館畠山
mournies - nearest hotel: Oasis
al jawf - nearest hotel: No hotel found
jiangkou - nearest hotel: 皇冠大酒店
yarmouth - nearest hotel: The Inn At Cape Cod
tvoroyri - nearest hotel: Hotel Tvøroyri
woodstock - nearest hotel: Holiday Inn Express
broome - nearest hotel: No hotel found
albany - nearest hotel: No hotel found
suez - nearest hotel: فندق لوسيندا
dadukou - nearest hotel: 锦江之星酒店
laguna - nearest hotel: Holiday Inn Express & Suites
jamestown - nearest hotel: DoubleTree Jamestown
westhill - nearest hotel: Holiday Inn Aberdeen West
zakynthos - nearest hotel: Phoenix Hotel
derzhavinsk - nearest hotel: Ишим
hasaki - nearest hotel: 大新旅館
ghosi - nearest hotel: No 

,City,Lat,Lng,Humidity,Country,Hotel Name
5,bethel,41.3712,-73.4140,48,US,Hampton Inn Danbury
6,kaspiyskiy,45.3929,47.3707,68,RU,Гостевой двор
19,eregli,37.5133,34.0467,47,TR,No hotel found
23,hamilton,39.1834,-84.5333,59,US,North Vista Manor
44,stanley,54.8680,-1.6985,74,GB,Hotel 52
...,...,...,...,...,...,...
550,chardara,41.2547,67.9692,36,KZ,Бақдаулет
551,tokmok,42.8417,75.3016,48,KG,Жибек
562,klipphausen,51.0833,13.5333,77,DE,Zur Ausspanne
566,moapa valley,36.5805,-114.4703,68,US,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [61]:
# Use the hover tooltip to add hotel and country to the hover message.
hover_cols = ["Hotel Name","Country"]

# Configure the plot
hotel_plot = hotel_df.hvplot.points("Lng",
                                   "Lat",
                                   geo = True,
                                   tiles = "OSM",
                                   frame_height= 700,
                                   frame_width= 900,
                                   size = "Humidity",
                                   color = "City",
                                   hover_cols = hover_cols)

# Display map
hotel_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)